In [1]:
#!pip install ccxt pandas tdqm numpy plotly python-dotenv nest-asyncio kaleido

In [2]:
import asyncio
import ccxt.async_support as ccxt
import pandas as pd
import numpy as np
from datetime import datetime 
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os
from dotenv import load_dotenv
import nest_asyncio
from tqdm import tqdm
import multiprocessing as mp
from concurrent.futures import ThreadPoolExecutor
from functools import partial

# Jupyter Notebook için asyncio desteğini etkinleştir
nest_asyncio.apply()

class TradingScanner:
    def __init__(self):
        load_dotenv()
        self.api_key = os.getenv('BINANCE_API_KEY')
        self.api_secret = os.getenv('BINANCE_API_SECRET')
        self.init_exchange_config()
        
        self.timeframes = ['15m', '1h', '4h', '1d']
        self.min_volume = 5_000_000
        
        self.timeframe_names = {
            '15m': '15 Dakika',
            '1h': '1 Saat',
            '4h': '4 Saat',
            '1d': '1 Gün'
        }
        
        self.timeframe_limits = {
            '15m': 200,
            '1h': 150,
            '4h': 150,
            '1d': 150
        }
        
        self.lookback_periods = {
            '15m': 96,
            '1h': 72,
            '4h': 60,
            '1d': 60
        }
        
        self.excluded_coins = {
            'USDC', 'USDT', 'USTC', 'FDUSD', 'BUSD', 
            'TUSD', 'DAI', 'UST', 'USDD', 'USDP', 'SUSD'
        }

    def init_exchange_config(self):
        """Exchange yapılandırmasını başlatır"""
        self.exchange_config = {
            'apiKey': self.api_key,
            'secret': self.api_secret,
            'enableRateLimit': True,
            'timeout': 30000,
            'options': {
                'defaultType': 'spot',
                'adjustForTimeDifference': True,
                'recvWindow': 60000
            }
        }

    def create_exchange(self):
        """Her process için yeni bir exchange instance'ı oluşturur"""
        return ccxt.binance(self.exchange_config)

    def calculate_squeeze(self, ohlcv, period=20):
        """TTM Squeeze hesaplama"""
        if len(ohlcv) < period:
            return False
            
        close = ohlcv[:, 4]
        high = ohlcv[:, 2]
        low = ohlcv[:, 3]
        
        middle = np.mean(close[-period:])
        stddev = np.std(close[-period:])
        bb_upper = middle + (2 * stddev)
        bb_lower = middle - (2 * stddev)
        
        tr = np.maximum.reduce([
            high[-period:] - low[-period:],
            np.abs(high[-period:] - np.roll(close[-period:], 1)),
            np.abs(low[-period:] - np.roll(close[-period:], 1))
        ])
        atr = np.mean(tr)
        kc_upper = middle + (1.5 * atr)
        kc_lower = middle - (1.5 * atr)
        
        return (bb_lower > kc_lower) and (bb_upper < kc_upper)

    def calculate_fibonacci_gradient(self, df, timeframe):
        """Fibonacci gradient hesaplama"""
        lookback = self.lookback_periods[timeframe]
        gradients = np.zeros(len(df))
        
        for i in range(lookback, len(df)):
            window = df.iloc[i-lookback:i]
            high = window['high'].astype(float)
            low = window['low'].astype(float)
            close = float(df['close'].iloc[i])
            
            fib_max = high.max()
            fib_min = low.min()
            max_min_diff = fib_max - fib_min
            
            if max_min_diff <= 0:
                continue
                
            fib_sell = fib_max - (max_min_diff * 0.382)
            fib_buy = fib_max - (max_min_diff * 0.707)
            
            if close >= fib_sell:
                sell_diff = ((close - fib_sell) / max_min_diff * 100)
                gradient = ((sell_diff * 2617.801) + 100000) / 2
            elif close <= fib_buy:
                buy_diff = ((fib_buy - close) / max_min_diff * 100)
                gradient = ((buy_diff * -2617.801) + 100000) / 2
            else:
                gradient = 50000
                
            gradients[i] = (100 - gradient/1000) * -1
            
        return gradients

    async def get_active_markets(self):
        """Aktif marketleri tarar ve filtreler"""
        exchange = self.create_exchange()
        try:
            print("\nAktif marketler taranıyor...")
            markets = await exchange.load_markets()
            active_pairs = []
            
            usdt_pairs = [s for s in markets if s.endswith('/USDT')]
            for symbol in tqdm(usdt_pairs[:50], desc="Market Taraması", ncols=100):  # İlk 50 market ile test
                try:
                    ticker = await exchange.fetch_ticker(symbol)
                    if ticker['quoteVolume'] >= self.min_volume:
                        active_pairs.append(symbol)
                except Exception as e:
                    continue
                    
                await asyncio.sleep(0.5)  # Rate limit için bekleme süresi arttırıldı
            
            return active_pairs
        finally:
            await exchange.close()

    async def process_market(self, symbol):
        """Tek bir market için analiz yapar"""
        exchange = self.create_exchange()
        try:
            results, ohlcv_data = await self.analyze_market(exchange, symbol)
            if results and symbol in results:
                return symbol, results[symbol], ohlcv_data
            return None
        finally:
            await exchange.close()

    async def analyze_market(self, exchange, symbol):
        """Market analizi yapan fonksiyon"""
        results = {}
        ohlcv_data = {}
        timeframe_data = {}
        
        for timeframe in self.timeframes:
            try:
                ohlcv = await exchange.fetch_ohlcv(
                    symbol, 
                    timeframe, 
                    limit=self.timeframe_limits[timeframe]
                )
                
                if not ohlcv or len(ohlcv) < self.lookback_periods[timeframe]:
                    print(f"Yetersiz veri: {symbol} - {timeframe}")
                    return results, ohlcv_data
                    
                timeframe_data[timeframe] = {
                    'ohlcv': ohlcv,
                    'array': np.array(ohlcv),
                    'df': pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
                }
                
            except Exception as e:
                print(f"Veri çekme hatası ({symbol} - {timeframe}): {str(e)}")
                return results, ohlcv_data
                
            await asyncio.sleep(0.5)

        all_signals = {}
        for timeframe, data in timeframe_data.items():
            gradients = self.calculate_fibonacci_gradient(data['df'], timeframe)
            last_gradient = gradients[-1]
            
            if last_gradient <= -70.7:
                signal = "Alış"
            elif last_gradient >= -38.2:
                signal = "Satış"
            else:
                return results, ohlcv_data
                
            all_signals[timeframe] = {
                'signal': signal,
                'gradient': last_gradient
            }
        
        first_signal = list(all_signals.values())[0]['signal']
        if not all(signal['signal'] == first_signal for signal in all_signals.values()):
            return results, ohlcv_data
            
        has_squeeze = False
        for timeframe, data in timeframe_data.items():
            if self.calculate_squeeze(data['array']):
                has_squeeze = True
                break
        
        if has_squeeze:
            results[symbol] = all_signals
            ohlcv_data = {tf: data['ohlcv'] for tf, data in timeframe_data.items()}
            
        return results, ohlcv_data

    def plot_analysis(self, symbol, data, ohlcv_data):
        """Grafik oluşturma fonksiyonu"""
        fig = make_subplots(
            rows=2, 
            cols=2, 
            subplot_titles=[f"{symbol} - {self.timeframe_names[tf]}" for tf in self.timeframes],
            vertical_spacing=0.1,
            horizontal_spacing=0.05
        )

        colors = {
            '15m': 'rgba(255, 0, 0, 0.3)', 
            '1h': 'rgba(0, 255, 0, 0.3)', 
            '4h': 'rgba(0, 0, 255, 0.3)',
            '1d': 'rgba(255, 165, 0, 0.3)'
        }

        for idx, timeframe in enumerate(self.timeframes):
            row = idx // 2 + 1
            col = idx % 2 + 1
            
            if timeframe in ohlcv_data:
                ohlcv = ohlcv_data[timeframe]
                df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
                df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
                
                gradients = self.calculate_fibonacci_gradient(df, timeframe)
                df['gradient'] = gradients
                
                fig.add_trace(
                    go.Candlestick(
                        x=df['timestamp'],
                        open=df['open'],
                        high=df['high'],
                        low=df['low'],
                        close=df['close'],
                        name=timeframe,
                        showlegend=False,
                        customdata=df['gradient'],
                        hovertext=[
                            f'Tarih: {date}<br>' +
                            f'Açılış: {open:.8f}<br>' +
                            f'Yüksek: {high:.8f}<br>' +
                            f'Düşük: {low:.8f}<br>' +
                            f'Kapanış: {close:.8f}<br>' +
                            f'Gradient: {gradient:.2f}'
                            for date, open, high, low, close, gradient in zip(
                                df['timestamp'], df['open'], df['high'], 
                                df['low'], df['close'], df['gradient']
                            )
                        ],
                        hoverinfo='text'
                    ),
                    row=row, col=col
                )

                for i in range(1, len(df)):
                    is_squeeze = self.calculate_squeeze(np.array(ohlcv[max(0, i-20):i+1]))
                    if is_squeeze:
                        fig.add_shape(
                            type="rect",
                            x0=df['timestamp'].iloc[i-1],
                            x1=df['timestamp'].iloc[i],
                            y0=df['low'].min(),
                            y1=df['high'].max(),
                            fillcolor=colors[timeframe],
                            opacity=0.3,
                            layer="below",
                            line_width=0,
                            row=row,
                            col=col
                        )

                if timeframe in data.get(symbol, {}):
                    signal_info = data[symbol][timeframe]
                    if signal_info['signal'] == "Alış":
                        fig.add_trace(
                            go.Scatter(
                                x=[df['timestamp'].iloc[-1]],
                                y=[df['low'].iloc[-1] * 0.999],
                                mode='markers',
                                marker=dict(symbol='triangle-up', size=12, color='lime'),
                                name='Alım Sinyali',
                                showlegend=idx == 0
                            ),
                            row=row, col=col
                        )
                    elif signal_info['signal'] == "Satış":
                        fig.add_trace(
                            go.Scatter(
                                x=[df['timestamp'].iloc[-1]],
                                y=[df['high'].iloc[-1] * 1.001],
                                mode='markers',
                                marker=dict(symbol='triangle-down', size=12, color='red'),
                                name='Satım Sinyali',
                                showlegend=idx == 0
                            ),
                            row=row, col=col
                        )

                fig.update_xaxes(
                    title_text="Tarih",
                    gridcolor='rgba(128,128,128,0.2)',
                    rangeslider=dict(visible=False),
                    row=row, 
                    col=col,
                    tickfont=dict(size=10)
                )
                fig.update_yaxes(
                    title_text="Fiyat",
                    gridcolor='rgba(128,128,128,0.2)',
                    row=row, 
                    col=col,
                    tickformat='.8f',
                    tickfont=dict(size=10)
                )

        # Genel grafik stili
        fig.update_layout(
            height=1200,
            width=1800,
            template="plotly_dark",
            paper_bgcolor='black',
            plot_bgcolor='black',
            title=dict(
                text=f"{symbol}",
                font=dict(size=28, color='white'),
                x=0.5,
                y=0.95
            ),
            showlegend=True,
            legend=dict(
                yanchor="top",
                y=0.99,
                xanchor="right",
                x=0.99,
                bgcolor='rgba(0,0,0,0.5)',
                font=dict(size=12, color='white')
            ),
            margin=dict(t=80, b=50, l=50, r=50)
        )

        # Subplot başlıkları için renk ayarı
        for annotation in fig['layout']['annotations']:
            annotation['font'] = dict(size=14, color='white')

        return fig

    async def run_analysis(self):
        """Ana analiz fonksiyonu - multiprocessing ile optimize edilmiş"""
        try:
            # Aktif marketleri al
            markets = await self.get_active_markets()
            print(f"\nToplam {len(markets)} aktif market bulundu")
            print(f"İşlem yapılacak marketler: {', '.join(markets)}\n")

            # CPU çekirdek sayısını al (1 çekirdek rezerve et)
            num_processes = max(1, mp.cpu_count() - 1)
            
            # Marketleri process sayısına göre böl
            market_chunks = self.chunk_markets(markets, num_processes)
            
            # ProcessPoolExecutor ile paralel işlem
            with concurrent.futures.ProcessPoolExecutor(max_workers=num_processes) as executor:
                # Her chunk için ayrı bir process başlat
                loop = asyncio.get_event_loop()
                futures = [
                    loop.run_in_executor(executor, 
                        lambda x: asyncio.run(self.run_parallel_analysis(x)), 
                        chunk)
                    for chunk in market_chunks
                ]
                
                # Tüm processlerin tamamlanmasını bekle
                results = []
                for future in tqdm(
                    concurrent.futures.as_completed(futures),
                    total=len(futures),
                    desc="Analiz İlerlemesi",
                    ncols=100
                ):
                    chunk_results = await future
                    results.extend([r for r in chunk_results if r is not None])

            # Sonuçları işle
            print("\n=== ÖZET ===")
            print(f"Toplam {len(results)} coin için sinyal bulundu:")
            
            for symbol, signals, ohlcv_data in results:
                print(f"\n{symbol} Sinyal Özeti:")
                for tf, result in signals.items():
                    print(f"✓ {self.timeframe_names[tf]}: {result['signal']} ({result['gradient']:.2f})")
                
                # Grafik oluştur
                fig = self.plot_analysis(symbol, {symbol: signals}, ohlcv_data)
                filename = f"{symbol.replace('/', '_')}_{datetime.now().strftime('%Y%m%d_%H%M')}.png"
                fig.write_image(filename, engine="kaleido", scale=2)

        except Exception as e:
            print(f"Hata oluştu: {str(e)}")
            raise e

    async def process_chunk(self, chunk):
        """Bir grup marketi işler"""
        tasks = []
        for symbol in chunk:
            tasks.append(self.process_market(symbol))
            await asyncio.sleep(0.1)  # Rate limit kontrolü
        return await asyncio.gather(*tasks, return_exceptions=True)

    async def run_analysis(self):
        """Ana analiz fonksiyonu"""
        try:
            markets = await self.get_active_markets()
            if not markets:
                print("Aktif market bulunamadı!")
                return

            print(f"\nToplam {len(markets)} aktif market bulundu")
            
            # Daha küçük chunk'lar kullan
            chunk_size = 5  # Chunk boyutu küçültüldü
            chunks = [markets[i:i + chunk_size] for i in range(0, len(markets), chunk_size)]
            
            results = []
            for chunk in tqdm(chunks, desc="Analiz İlerlemesi", ncols=100):
                try:
                    chunk_results = await self.process_chunk(chunk)
                    valid_results = [r for r in chunk_results if r is not None and not isinstance(r, Exception)]
                    results.extend(valid_results)
                    await asyncio.sleep(1)  # Chunklar arası bekleme
                except Exception as e:
                    print(f"Chunk işleme hatası: {str(e)}")
                    continue

            print("\n=== ÖZET ===")
            print(f"Toplam {len(results)} coin için sinyal bulundu:")
            
            for symbol, signals, ohlcv_data in results:
                try:
                    print(f"\n{symbol} Sinyal Özeti:")
                    for tf, result in signals.items():
                        print(f"✓ {self.timeframe_names[tf]}: {result['signal']} ({result['gradient']:.2f})")
                    
                    fig = self.plot_analysis(symbol, {symbol: signals}, ohlcv_data)
                    filename = f"{symbol.replace('/', '_')}_{datetime.now().strftime('%Y%m%d_%H%M')}.png"
                    fig.write_image(filename, engine="kaleido", scale=2)
                    
                    # Belleği temizle
                    del fig
                except Exception as e:
                    print(f"{symbol} için grafik oluşturma hatası: {str(e)}")
                    continue

        except Exception as e:
            print(f"Genel hata: {str(e)}")
            import traceback
            traceback.print_exc()

    @classmethod
    async def create_and_run(cls):
        """Sınıfın instance'ını oluşturup çalıştıran factory metod"""
        scanner = cls()
        await scanner.run_analysis()
        return scanner

def run_scanner():
    """Jupyter için çalıştırma fonksiyonu"""
    try:
        scanner = TradingScanner()
        asyncio.run(scanner.run_analysis())
    except Exception as e:
        print(f"\nBir hata oluştu: {e}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    run_scanner()


Aktif marketler taranıyor...


Market Taraması: 100%|██████████████████████████████████████████████| 50/50 [00:37<00:00,  1.34it/s]



Toplam 15 aktif market bulundu


Analiz İlerlemesi: 100%|██████████████████████████████████████████████| 3/3 [00:31<00:00, 10.48s/it]



=== ÖZET ===
Toplam 2 coin için sinyal bulundu:

FET/USDT Sinyal Özeti:
✓ 15 Dakika: Alış (-77.64)
✓ 1 Saat: Alış (-83.48)
✓ 4 Saat: Alış (-84.11)
✓ 1 Gün: Alış (-71.66)

ALGO/USDT Sinyal Özeti:
✓ 15 Dakika: Alış (-83.56)
✓ 1 Saat: Alış (-88.35)
✓ 4 Saat: Alış (-88.35)
✓ 1 Gün: Alış (-101.40)
